In [25]:
import sys
print(sys.path)
sys.path.insert(0,'/data/ssd-caffe/py2_caffe/python/')
import caffe  
import numpy as np
from collections import OrderedDict
from ConfigParser import ConfigParser

['/data/ssd-caffe/py2_caffe/python/', '/data/ssd-caffe/py2_caffe/python/', '/data/ssd-caffe/py2_caffe/python/', '', '/data/github_repos/yolov3-tiny-fit-ncs/ncs/convert_tools', '/data/ssd-caffe/caffe/python', '/data/ssd-caffe/new-yolov3-caffe/python', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-x86_64-linux-gnu', '/usr/lib/python2.7/lib-tk', '/usr/lib/python2.7/lib-old', '/usr/lib/python2.7/lib-dynload', '/home/yyp/.local/lib/python2.7/site-packages', '/usr/local/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages/gtk-2.0', '/home/yyp/.local/lib/python2.7/site-packages/IPython/extensions', '/home/yyp/.ipython']


In [26]:
class UniqDict(OrderedDict):
    _unique = 0
    def __setitem__(self, key, val):
        if isinstance(val, OrderedDict):
            self._unique += 1
            key += "_"+str(self._unique)
        OrderedDict.__setitem__(self, key, val)

In [27]:
def parse_cfg(cfgfile):
    parser = ConfigParser(dict_type=UniqDict)
    parser.read(cfgfile)
    blocks = parser.sections()
    return blocks, parser
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [34]:
def cfg2prototxt(cfgfile):
    blocks, parser = parse_cfg(cfgfile)
    print(blocks)
    layers = []  
    props = OrderedDict()  
    bottom = 'data'  
    layer_id = 1  
    topnames = dict()  
    for block in blocks:
        items = dict(parser.items(block))
        items['type'] = block.split('_')[0]
        print(items)
        if items['type'] == 'net':  
            props['name'] = 'Darkent2Caffe'  
            props['input'] = 'data'  
            props['input_dim'] = ['1']  
            props['input_dim'].append(items['channels'])  
            props['input_dim'].append(items['height'])  
            props['input_dim'].append(items['width'])  
            continue  
        elif (items['type'] == 'convolutional' or items['type'] == 'deconvolutional'):  
            conv_layer = OrderedDict()  
            conv_layer['bottom'] = bottom  
            if items.has_key('name'):  
                conv_layer['top'] = items['name']  
                conv_layer['name'] = items['name']  
            else:  
                conv_layer['top'] = 'layer%d-conv' % layer_id  
                conv_layer['name'] = 'layer%d-conv' % layer_id
            if(items['type'] == 'deconvolutional'):
                conv_layer['type'] = 'Deconvolution'
            elif(items['type'] == 'convolutional'):
                conv_layer['type'] = 'Convolution' 
            convolution_param = OrderedDict()  
            convolution_param['num_output'] = items['filters']  
            convolution_param['kernel_size'] = items['size']  
            if items['pad'] == '1':  
                convolution_param['pad'] = str(int(convolution_param['kernel_size'])/2)  
            convolution_param['stride'] = items['stride']
            if items.has_key('batch_normalize'):
                if items['batch_normalize'] == '1':  
                    convolution_param['bias_term'] = 'false' 
            else:  
                convolution_param['bias_term'] = 'true'  
            conv_layer['convolution_param'] = convolution_param  
            layers.append(conv_layer)  
            bottom = conv_layer['top']  
            if items.has_key('batch_normalize'):
                if items['batch_normalize'] == '1': 
                    bn_layer = OrderedDict()  
                    bn_layer['bottom'] = bottom  
                    bn_layer['top'] = bottom  
                    if items.has_key('name'):  
                        bn_layer['name'] = '%s-bn' % items['name']  
                    else:  
                        bn_layer['name'] = 'layer%d-bn' % layer_id  
                    bn_layer['type'] = 'BatchNorm'  
                    batch_norm_param = OrderedDict()  
                    batch_norm_param['use_global_stats'] = 'true'  
                    bn_layer['batch_norm_param'] = batch_norm_param  
                    layers.append(bn_layer)  

                    scale_layer = OrderedDict()  
                    scale_layer['bottom'] = bottom  
                    scale_layer['top'] = bottom  
                    if items.has_key('name'):  
                        scale_layer['name'] = '%s-scale' % items['name']  
                    else:  
                        scale_layer['name'] = 'layer%d-scale' % layer_id  
                    scale_layer['type'] = 'Scale'  
                    scale_param = OrderedDict()  
                    scale_param['bias_term'] = 'true'  
                    scale_layer['scale_param'] = scale_param  
                    layers.append(scale_layer)  

            if items['activation'] != 'linear':  
                relu_layer = OrderedDict()  
                relu_layer['bottom'] = bottom  
                relu_layer['top'] = bottom  
                if items.has_key('name'):  
                    relu_layer['name'] = '%s-act' % items['name']  
                else:  
                    relu_layer['name'] = 'layer%d-act' % layer_id  
                relu_layer['type'] = 'ReLU'  
                if items['activation'] == 'leaky':  
                    relu_param = OrderedDict()  
                    relu_param['negative_slope'] = '0.1'  
                    relu_layer['relu_param'] = relu_param  
                layers.append(relu_layer)  
            topnames[layer_id] = bottom  
            layer_id = layer_id+1  
        elif items['type'] == 'maxpool':  
            max_layer = OrderedDict()  
            max_layer['bottom'] = bottom  
            if items.has_key('name'):  
                max_layer['top'] = items['name']  
                max_layer['name'] = items['name']  
            else:  
                max_layer['top'] = 'layer%d-maxpool' % layer_id  
                max_layer['name'] = 'layer%d-maxpool' % layer_id  
            max_layer['type'] = 'Pooling'  
            pooling_param = OrderedDict()  
            pooling_param['kernel_size'] = items['size']  
            pooling_param['stride'] = items['stride']  
            pooling_param['pool'] = 'MAX'  
            if items.has_key('pad') and int(items['pad']) == 1:  
                pooling_param['pad'] = str((int(items['size'])-1)/2)  
            max_layer['pooling_param'] = pooling_param  
            layers.append(max_layer)  
            bottom = max_layer['top']  
            topnames[layer_id] = bottom  
            layer_id = layer_id+1  
        elif items['type'] == 'avgpool':  
            avg_layer = OrderedDict()  
            avg_layer['bottom'] = bottom  
            if block.has_key('name'):  
                avg_layer['top'] = items['name']  
                avg_layer['name'] = items['name']  
            else:  
                avg_layer['top'] = 'layer%d-avgpool' % layer_id  
                avg_layer['name'] = 'layer%d-avgpool' % layer_id  
            avg_layer['type'] = 'Pooling'  
            pooling_param = OrderedDict()  
            pooling_param['kernel_size'] = 7  
            pooling_param['stride'] = 1  
            pooling_param['pool'] = 'AVE'  
            avg_layer['pooling_param'] = pooling_param  
            layers.append(avg_layer)  
            bottom = avg_layer['top']  
            topnames[layer_id] = bottom  
            layer_id = layer_id+1  
        elif items['type'] == 'yolo': 
            layer_id = layer_id + 1
            continue
            region_layer = OrderedDict()  
            region_layer['bottom'] = bottom  
            if items.has_key('name'):  
                region_layer['top'] = items['name']  
                region_layer['name'] = items['name']  
            else:  
                region_layer['top'] = 'layer%d-yolo' % layer_id  
                region_layer['name'] = 'layer%d-yolo' % layer_id  
            region_layer['type'] = 'Yolo'  
            region_param = OrderedDict()  
            region_param['anchors'] = items['anchors'].strip()  
            region_param['classes'] = items['classes']  
            region_param['num'] = items['num']  
            region_layer['yolo_param'] = region_param  
            layers.append(region_layer)  
            bottom = region_layer['top']  
            topnames[layer_id] = bottom  
            layer_id = layer_id + 1

        elif items['type'] == 'route':
            route_layer = OrderedDict()  
            layer_name = str(items['layers']).split(',')  
            print(layer_name[0])  
            bottom_layer_size = len(str(items['layers']).split(','))  
        #print(bottom_layer_size)  
            if(1 == bottom_layer_size):  
                prev_layer_id = layer_id + int(items['layers'])  
                bottom = topnames[prev_layer_id]  
                #topnames[layer_id] = bottom  
            route_layer['bottom'] = bottom  
            if(2 == bottom_layer_size):  
                prev_layer_id1 = layer_id + int(layer_name[0])  
                #print(prev_layer_id1)  
                prev_layer_id2 = int(layer_name[1]) + 1  
                print(topnames)  
                bottom1 = topnames[prev_layer_id1]  
                bottom2 = topnames[prev_layer_id2]  
                route_layer['bottom'] = [bottom1, bottom2]  
            if items.has_key('name'):  
                route_layer['top'] = items['name']  
                route_layer['name'] = items['name']  
            else:  
                route_layer['top'] = 'layer%d-route' % layer_id  
                route_layer['name'] = 'layer%d-route' % layer_id  
            route_layer['type'] = 'Concat'
            print(route_layer)  
            layers.append(route_layer)  
            bottom = route_layer['top']  
            print(layer_id)  
            topnames[layer_id] = bottom  
            layer_id = layer_id + 1  

        elif items['type'] == 'upsample':
            upsample_layer = OrderedDict()  
            print(items['stride'])  
            upsample_layer['bottom'] = bottom  
            if items.has_key('name'):  
                upsample_layer['top'] = items['name']  
                upsample_layer['name'] = items['name']  
            else:  
                upsample_layer['top'] = 'layer%d-upsample' % layer_id  
                upsample_layer['name'] = 'layer%d-upsample' % layer_id  
            upsample_layer['type'] = 'Upsample'  
            upsample_param = OrderedDict()  
            upsample_param['scale'] = items['stride']  
            upsample_layer['upsample_param'] = upsample_param  
            print(upsample_layer)  
            layers.append(upsample_layer)  
            bottom = upsample_layer['top']  
            print('upsample:',layer_id)  
            topnames[layer_id] = bottom  
            layer_id = layer_id + 1  

        elif items['type'] == 'shortcut':  
            prev_layer_id1 = layer_id + int(items['from'])  
            prev_layer_id2 = layer_id - 1  
            bottom1 = topnames[prev_layer_id1]  
            bottom2= topnames[prev_layer_id2]  
            shortcut_layer = OrderedDict()  
            shortcut_layer['bottom'] = [bottom1, bottom2]  
            if items.has_key('name'):  
                shortcut_layer['top'] = items['name']  
                shortcut_layer['name'] = items['name']  
            else:  
                shortcut_layer['top'] = 'layer%d-shortcut' % layer_id  
                shortcut_layer['name'] = 'layer%d-shortcut' % layer_id  
            shortcut_layer['type'] = 'Eltwise'
            eltwise_param = OrderedDict()  
            eltwise_param['operation'] = 'SUM'  
            shortcut_layer['eltwise_param'] = eltwise_param  
            layers.append(shortcut_layer)  
            bottom = shortcut_layer['top']  

            if items['activation'] != 'linear':  
                relu_layer = OrderedDict()  
                relu_layer['bottom'] = bottom  
                relu_layer['top'] = bottom  
                if block.has_key('name'):  
                    relu_layer['name'] = '%s-act' % items['name']  
                else:  
                    relu_layer['name'] = 'layer%d-act' % layer_id  
                relu_layer['type'] = 'ReLU'  
                if items['activation'] == 'leaky':  
                    relu_param = OrderedDict()  
                    relu_param['negative_slope'] = '0.1'  
                    relu_layer['relu_param'] = relu_param  
                layers.append(relu_layer)  
            topnames[layer_id] = bottom  
            layer_id = layer_id + 1             

        elif items['type'] == 'connected':  
            fc_layer = OrderedDict()  
            fc_layer['bottom'] = bottom  
            if items.has_key('name'):  
                fc_layer['top'] = items['name']  
                fc_layer['name'] = items['name']  
            else:  
                fc_layer['top'] = 'layer%d-fc' % layer_id  
                fc_layer['name'] = 'layer%d-fc' % layer_id  
            fc_layer['type'] = 'InnerProduct'  
            fc_param = OrderedDict()  
            fc_param['num_output'] = int(items['output'])  
            fc_layer['inner_product_param'] = fc_param  
            layers.append(fc_layer)  
            bottom = fc_layer['top']  

            if items['activation'] != 'linear':  
                relu_layer = OrderedDict()  
                relu_layer['bottom'] = bottom  
                relu_layer['top'] = bottom  
                if items.has_key('name'):  
                    relu_layer['name'] = '%s-act' % items['name']  
                else:  
                    relu_layer['name'] = 'layer%d-act' % layer_id  
                relu_layer['type'] = 'ReLU'  
                if items['activation'] == 'leaky':  
                    relu_param = OrderedDict()  
                    relu_param['negative_slope'] = '0.1'  
                    relu_layer['relu_param'] = relu_param  
                layers.append(relu_layer)  
            topnames[layer_id] = bottom  
            layer_id = layer_id+1  
        else:  
            print('unknow layer type %s ' % items['type'])  
            topnames[layer_id] = bottom  
            layer_id = layer_id + 1  

    net_info = OrderedDict()  
    net_info['props'] = props  
    net_info['layers'] = layers  
    return net_info 
def save_prototxt(net_info, protofile, yolo=False):
    fp = open(protofile, 'w')
    # whether add double quote
    def format_value(value):
        #str = u'%s' % value
        #if str.isnumeric():
        if is_number(value):
            return value
        elif value == 'true' or value == 'false' or value == 'MAX' or value == 'SUM' or value == 'AVE':
            return value
        else:
            return '\"%s\"' % value

    def print_block(block_info, prefix, indent):
        blanks = ''.join([' ']*indent)
        print >>fp, '%s%s {' % (blanks, prefix)
        for key,value in block_info.items():
            if type(value) == OrderedDict:
                print_block(value, key, indent+4)
            elif type(value) == list:
                for v in value:
                    print >> fp, '%s    %s: %s' % (blanks, key, format_value(v))
            else:
                print >> fp, '%s    %s: %s' % (blanks, key, format_value(value))
        print >> fp, '%s}' % blanks
        
    props = net_info['props']
    layers = net_info['layers']
    print >> fp, 'name: \"%s\"' % props['name']
    print >> fp, 'input: \"%s\"' % props['input']
    print >> fp, 'input_dim: %s' % props['input_dim'][0]
    print >> fp, 'input_dim: %s' % props['input_dim'][1]
    print >> fp, 'input_dim: %s' % props['input_dim'][2]
    print >> fp, 'input_dim: %s' % props['input_dim'][3]
    print >> fp, ''
    for layer in layers:
        if layer['type'] != 'yolo' or yolo == True:
            print_block(layer, 'layer', 0)
    fp.close()

In [37]:
#cfgfile = '/data/github_repos/darknet/ncs/yolov3-tiny-ncs-without-last-maxpool.cfg'
cfgfile = '/data/darknet/cfg/yolov3.cfg'
#cfg2prototxt(cfgfile)

In [38]:
if __name__ == '__main__':  
    '''
    cfgfile = sys.argv[1]   
    weightfile = sys.argv[2]  
    protofile = sys.argv[3]  
    caffemodel = sys.argv[4]
    '''
    import datetime
    
    saved_prototxt = str(datetime.datetime.now()).split(' ')[-1] + cfgfile.split('/')[-1] + '.prototxt'
    net_info = cfg2prototxt(cfgfile)    
    save_prototxt(net_info, saved_prototxt) 
    #darknet2caffe(cfgfile, weightfile, protofile, caffemodel)

['net_1', 'convolutional_2', 'convolutional_3', 'convolutional_4', 'convolutional_5', 'shortcut_6', 'convolutional_7', 'convolutional_8', 'convolutional_9', 'shortcut_10', 'convolutional_11', 'convolutional_12', 'shortcut_13', 'convolutional_14', 'convolutional_15', 'convolutional_16', 'shortcut_17', 'convolutional_18', 'convolutional_19', 'shortcut_20', 'convolutional_21', 'convolutional_22', 'shortcut_23', 'convolutional_24', 'convolutional_25', 'shortcut_26', 'convolutional_27', 'convolutional_28', 'shortcut_29', 'convolutional_30', 'convolutional_31', 'shortcut_32', 'convolutional_33', 'convolutional_34', 'shortcut_35', 'convolutional_36', 'convolutional_37', 'shortcut_38', 'convolutional_39', 'convolutional_40', 'convolutional_41', 'shortcut_42', 'convolutional_43', 'convolutional_44', 'shortcut_45', 'convolutional_46', 'convolutional_47', 'shortcut_48', 'convolutional_49', 'convolutional_50', 'shortcut_51', 'convolutional_52', 'convolutional_53', 'shortcut_54', 'convolutional_55'